In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go
import FundamentalAnalysis as fa
import numpy as np
import datetime as dt
import time 
import pickle

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_table
import os.path
from os import path

FA_API_KEY = 'b5e24ca6cc11c806a29842821921ad85'
QTR_LOOKUP_TABLE = {'01': 'Q1',
                    '02': 'Q1',
                    '03': 'Q1',
                    '04': 'Q2',
                    '05': 'Q2',
                    '06': 'Q2',
                   '07': 'Q3', 
                    '08': 'Q3',
                    '09': 'Q3',
                    '10': 'Q4',
                    '11': 'Q4',
                   '12': 'Q4'}
QTR_DATES = [pd.to_datetime('2020-01-01'),
            pd.to_datetime('2020-03-31'), 
            pd.to_datetime('2020-06-30'), 
            pd.to_datetime('2020-09-30')]

In [2]:
def get_quarterly_dates(df):
    qtr_dates = []
    for idx in df.index:
        split_date = str(idx).split('-')
        qtr_date = split_date[0] + '-' + QTR_LOOKUP_TABLE[split_date[1]]
        qtr_dates.append(qtr_date)
    return qtr_dates

In [3]:
def get_fundamentals(ticker) -> pd.DataFrame:
    # balance_sheet_quarterly = fa.balance_sheet_statement(ticker, FA_API_KEY, period="quarterly")
    # cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, FA_API_KEY, period="quarter")
    # key_metrics = fa.key_metrics(ticker, FA_API_KEY, period="quarter")

    financial_ratios_quarterly = fa.financial_ratios(ticker, FA_API_KEY, period="quarter")
    financial_statement_growth = fa.financial_statement_growth(ticker, FA_API_KEY, period="quarter")

    roe = financial_ratios_quarterly.loc['returnOnEquity']
    gross_profit_margin = financial_ratios_quarterly.loc['grossProfitMargin'] 
    operating_profit_margin = financial_ratios_quarterly.loc['operatingProfitMargin'] 
    pretaxProfitMargin = financial_ratios_quarterly.loc['pretaxProfitMargin']
    netProfitMargin = financial_ratios_quarterly.loc['netProfitMargin']

    revenue_growth = financial_statement_growth.loc['revenueGrowth']
    gross_profit_growth = financial_statement_growth.loc['grossProfitGrowth']
    ebit_growth = financial_statement_growth.loc['ebitgrowth']
    quarterly_dates = get_quarterly_dates(roe)
    fmtls_df = pd.DataFrame.from_dict({'qtr_dates': quarterly_dates,
                             'roe': roe, 
                             'grossProfitMargin': gross_profit_margin, 
                             'operatingProfitMargin': operating_profit_margin,
                             'pretaxProfitMargin': pretaxProfitMargin,
                             'netProfitMargin': netProfitMargin,
                             'revenueGrowth': revenue_growth,
                             'ebitGrowth': ebit_growth, 
                             'grossProfitGrowth': gross_profit_growth}, orient='index').T.dropna()

#     fmtls_df.index = pd.to_datetime(fmtls_df.index)
    fmtls_df = fmtls_df.iloc[0:10]
    fmtls_df = fmtls_df.sort_index()
    fmtls_df.index = fmtls_df['qtr_dates']
    fmtls_df.drop(labels=['qtr_dates'], axis=1, inplace=True)
    fmtls_df = fmtls_df.round(3)

    return fmtls_df

In [4]:
def get_technicals(ticker, threshold=0.10):
    weekly_stock_data = fa.stock_data(ticker, period='1y', interval='1wk')
    daily_stock_data = fa.stock_data(ticker, period='1y', interval='1d')
    daily_stock_data.index = pd.to_datetime(daily_stock_data.index)

    weekly_stock_data = weekly_stock_data[['close', 'open']]
    weekly_stock_data['next_week_gain'] = (weekly_stock_data.loc[:,'close'].shift(-1) / weekly_stock_data.loc[:,'close']) - 1
    weekly_stock_data.dropna(inplace=True)

    wks_of_interest = weekly_stock_data[abs(weekly_stock_data['next_week_gain']) > threshold]
    wks_of_interest.index = pd.to_datetime(wks_of_interest.index)
    wks_of_interest = wks_of_interest[wks_of_interest.index.year == 2020]
    wks_of_interest['int_idx'] = np.linspace(1, len(wks_of_interest), num=len(wks_of_interest))
    return {'weekly': weekly_stock_data, 
            'daily': daily_stock_data, 
            'interesting_weeks': wks_of_interest}

In [5]:
stocks = ['TSLA', 'AAPL', 'AMZN', 'VEEV']
fundamentals_df = {}
technicals_df = {}
if not path.exists("technicals_df"):
    print("couldn't find pickled data")
    for ticker in stocks: 
        fundamentals_df[ticker] = get_fundamentals(ticker)
        technicals_df[ticker] = get_technicals(ticker)
else: 
    with open('fundamentals_df', 'rb') as handle: 
        fundamentals_df = pickle.load(handle)
        for ticker in stocks: 
            fundamentals_df[ticker] = fundamentals_df[ticker].astype(float)
            fundamentals_df[ticker] = fundamentals_df[ticker]*100
            fundamentals_df[ticker] = fundamentals_df[ticker].round(2)
    with open('technicals_df', 'rb') as handle: 
        technicals_df = pickle.load(handle)
    print("Found and loaded pickled data!\nDidn't query FundamentalAnalysis API")
    

Found and loaded pickled data!
Didn't query FundamentalAnalysis API


In [6]:
table_columns = fundamentals_df[ticker].columns

In [7]:
app = JupyterDash(__name__)

In [8]:
# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("CHACHINNGGGG MUTHA FUCKKKAAASSSSS", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "TSLA", "value": 'TSLA'},
                     {"label": "AAPL", "value": 'AAPL'},
                     {"label": "AMZN", "value": 'AMZN'},
                     {"label": "VEEV", "value": 'VEEV'}],
                 multi=False,
                 value='TSLA',
                 style={'width': "40%"},
                 searchable=False
                 ),
    html.Br(),
    html.Div(id='output_container', children=[]),
    html.Br(),
    dash_table.DataTable(
        id='fundamental_table',
        columns=[{'name': i, 'id':i} for i in table_columns],
        data=[],
        editable=True),
    html.Br(),
    dcc.Graph(id='fundamental_line_graph', figure={}),
    html.Br(),
    dcc.Graph(id='technical_line_graph', figure={})

])

In [9]:
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='fundamental_line_graph', component_property='figure'),
     Output(component_id='technical_line_graph', component_property='figure'),
     Output(component_id='fundamental_table', component_property='data')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    container = f'You selected {option_slctd}'

    fund_graph_df = fundamentals_df[option_slctd]
    print(fund_graph_df)
    # Fundamentals Table
    fund_table = fundamentals_df[option_slctd].to_dict('records')

    
    # Fundamentals Line Graph
    fundamental_line_graph = px.line(fund_graph_df)
    fundamental_line_graph.update_layout(
        title=f"{option_slctd} Fundamental Stuff",
        xaxis_title="Date",
        yaxis_title="Percent [%]",
        autosize=True
    )
    
    # Technicals Line Graph
    slctd_weekly = technicals_df[option_slctd]['weekly']
    slctd_daily = technicals_df[option_slctd]['daily']
    slctd_important_weeks = technicals_df[option_slctd]['interesting_weeks']
    
    technical_line_graph = px.line(x=slctd_weekly['close'].index, 
              y=slctd_weekly['close'])
    technical_line_graph.add_trace(go.Scatter(x=slctd_important_weeks.index, 
                             y=slctd_important_weeks['close'][slctd_important_weeks.index],
                             text=slctd_important_weeks['int_idx'],
                             mode='markers+text',
                             textposition="top center",
                             name='Big Weeks'))

    for i in range(1,4):
        technical_line_graph.add_vline(x=QTR_DATES[i])

    technical_line_graph.update_xaxes(type='date')
    technical_line_graph.update_layout(
        title=f"{option_slctd} Technical Stuff",
        xaxis_title="Date",
        yaxis_title="Price [$]",
        autosize=True
    )
    technical_line_graph.update_xaxes(tickangle=45)
    

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    return container, fundamental_line_graph, technical_line_graph, fund_table

In [10]:
# ------------------------------------------------------------------------------
app.run_server(port=8051)

Dash app running on http://127.0.0.1:8051/
